In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
R.EnableImplicitMT()
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

Welcome to JupyROOT 6.29/01
Recompile is not needed


'Utility Functions V1.0.4 \n'

In [2]:
%%time
root_file = R.TFile("z_endpoint_plots.root","RECREATE")
energies=["0.5","1.0","1.5","2.0","2.5","3.0","3.5","3.742"]
particle="e-"
particle_name="electron"

histos_primary = []
histos_daughter = []

for energy in energies:
    ch = R.TChain("MiniDST")
    if particle == "e+":
        particle_s = "e\+"
    else:
        particle_s = particle
    ch.Add("/data/HPS/data/physrun2021/MC/hpsForward_"+particle_s+"_"+energy+"GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
    # ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
    print(f"E={energy} type={particle} Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")
    df = R.RDataFrame(ch)
    dfx = df.Define("n_kf_track","int n=0; for(int i=0;i<track_type.size();++i){if(track_type[i]==1) n+=1;}; return n") \
        .Filter("n_kf_track==1").Filter("ecal_cluster_energy.size()==1") \
        .Define("trk_idx","for(int i=0;i<track_type.size();++i){if(track_type[i]==1) return(i);} return(-1);") \
        .Define("diff_posx_ecal","ecal_cluster_x[0] - track_x_at_ecal[trk_idx]") \
        .Define("diff_posy_ecal","ecal_cluster_y[0] - track_y_at_ecal[trk_idx]") \
        .Define("diff_E_p","part_energy[0] - get_vec_abs(part_px,part_py,part_pz)[0]") \
        .Define("diff_p_kf_gbl","get_vec_abs(part_px,part_py,part_pz)[0] - get_vec_abs(part_px,part_py,part_pz)[1]") \
        .Define("mc_part_primary_z","find_end_z_of_primary_mc_part(mc_part_z, mc_part_end_z)") \
        .Define("mc_part_daughters_z","find_average_end_of_daughters_of_primary_mc_part(mc_part_z, mc_part_end_z, mc_part_daughters)")
    cnt  = df.Count()
    cntx = dfx.Count()
    hx_mc_part_primary_z = dfx.Histo1D((f"hx_mc_part_primary_z_{particle_name}_{energy}",f"End of track for primary MC {particle} E={energy}.;z[mm]",1000,1400.,1700.),"mc_part_primary_z")
    hx_mc_part_ave_daughter_z = dfx.Histo1D((f"hx_mc_part_ave_daughter_z_{particle_name}_{energy}",f"Average end of track for primary MC daughters {particle} E={energy};z[mm]",1000,1400.,1700.),"mc_part_daughters_z")
    hx_mc_part_primary_z.GetPtr().Write()           # Write the histogram to the file.
    hx_mc_part_primary_z.GetPtr().SetDirectory(0)   # Dis-associate the histogram with the file, so it does not get destroyed when the file closes.
    hx_mc_part_ave_daughter_z.Write()
    hx_mc_part_ave_daughter_z.SetDirectory(0)
    histos_primary.append(hx_mc_part_primary_z.GetPtr().Clone())
    histos_daughter.append(hx_mc_part_ave_daughter_z.GetPtr().Clone())
root_file.ls()
root_file.Write()
# root_file.Close()

E=0.5 type=e- Number of events loaded:   0.134M
E=1.0 type=e- Number of events loaded:   0.687M
E=1.5 type=e- Number of events loaded:   1.125M
E=2.0 type=e- Number of events loaded:   1.457M
E=2.5 type=e- Number of events loaded:   1.458M
E=3.0 type=e- Number of events loaded:   1.243M
E=3.5 type=e- Number of events loaded:   1.312M
E=3.742 type=e- Number of events loaded:   2.007M
CPU times: user 56 s, sys: 3.65 s, total: 59.6 s
Wall time: 15.2 s


37726

TFile**		z_endpoint_plots.root	
 TFile*		z_endpoint_plots.root	
  OBJ: TH1D	hx_mc_part_primary_z_electron_0.5	End of track for primary MC e- E=0.5. : 0 at: 0x169a36ba0
  OBJ: TH1D	hx_mc_part_ave_daughter_z_electron_0.5	Average end of track for primary MC daughters e- E=0.5 : 0 at: 0x169a38300
  OBJ: TH1D	hx_mc_part_primary_z_electron_1.0	End of track for primary MC e- E=1.0. : 0 at: 0x17de0d730
  OBJ: TH1D	hx_mc_part_ave_daughter_z_electron_1.0	Average end of track for primary MC daughters e- E=1.0 : 0 at: 0x17de0db20
  OBJ: TH1D	hx_mc_part_primary_z_electron_1.5	End of track for primary MC e- E=1.5. : 0 at: 0x168689c10
  OBJ: TH1D	hx_mc_part_ave_daughter_z_electron_1.5	Average end of track for primary MC daughters e- E=1.5 : 0 at: 0x16868a000
  OBJ: TH1D	hx_mc_part_primary_z_electron_2.0	End of track for primary MC e- E=2.0. : 0 at: 0x17de393f0
  OBJ: TH1D	hx_mc_part_ave_daughter_z_electron_2.0	Average end of track for primary MC daughters e- E=2.0 : 0 at: 0x17de397e0
  OBJ: TH1D	hx_m

In [3]:
h_colors = [R.kViolet, R.kCyan, R.kBlue, R.kGreen, R.kOrange, R.kRed, R.kBlack, R.kGray]
cc5 = R.TCanvas("cc5","Canvas",800,600)
histos_daughter[7].SetTitle("Average end of track z for e- daughter particles.")
histos_daughter[7].Draw()
lines = []
leg5 = R.TLegend(0.71,0.6,0.89,0.89)
mean = 0.
for i in range(len(histos_daughter)):
    hhd = histos_daughter[i]
    hhd.SetStats(0)
    hhd.SetLineColor(h_colors[i])
    hhd.SetLineWidth(2)
    hhd.Draw("same")
    mean += hhd.GetMean()
    leg5.AddEntry(hhd, f"E={energies[i]}")
mean = mean/len(histos_daughter)
l1 = R.TLine(1394, 0, 1394, 1.05*histos_daughter[7].GetMaximum())
l1.SetLineWidth(2)
l1.SetLineColor(R.kGreen+1)
l1.Draw()
l2 = R.TLine(mean, 0, mean, 1.05*histos_daughter[7].GetMaximum())
l2.SetLineWidth(2)
l2.SetLineColor(R.kRed+1)
l2.Draw()
leg5.AddEntry(l1,"Ecal face.")
leg5.AddEntry(l2,"Ecal ave z depth")
leg5.Draw()
cc5.Draw()
print(f"The mean of all the histograms is at {mean}")

The mean of all the histograms is at 1474.2202599917505


In [18]:
cc5.SaveAs("ECal_z_distribution.pdf")

Info in <TCanvas::Print>: pdf file ECal_z_distribution.pdf has been created


In [10]:
R.gInterpreter.Declare('''
void setup_langau(void){
auto langau = new TF1("langau",langaufun,1400.,1700.,4);
langau->SetParameters(3., 1450., 1000., 10.);
langau->SetParNames("Width","Mu","Area","Sigma");
std::cout << "The langau function is now setup. " << std::endl;
}
''')
R.setup_langau()

input_line_188:2:6: error: redefinition of 'setup_langau'
void setup_langau(void){
     ^
input_line_182:2:6: note: previous definition is here
void setup_langau(void){
     ^


In [14]:
print(f"Mean original: {histos_daughter[7].GetMean()}")
cc6 = R.TCanvas("cc6","Canvas",800,600)
#cc6.SetLogy()
#histos_daughter[7].GetXaxis().SetRangeUser(1440, 1600)
histos_daughter[7].Draw()
histos_daughter[7].Fit("langau")
R.langau.Draw("same")
#ffun.Draw("same")
cc6.Draw()
print(f"Mean zoomed : {histos_daughter[7].GetMean()}")

Mean original: 1472.5259114214193
Mean zoomed : 1472.5259114214193
 FCN=122564 FROM MIGRAD    STATUS=CONVERGED      60 CALLS          61 TOTAL
                     EDM=2.87774e-09    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Width        3.82989e+00   1.06526e-02   1.12852e-03   4.08712e-03
   2  Mu           1.46179e+03   2.38715e-02   2.10014e-03   7.22957e-04
   3  Area         2.90091e+05   2.97917e+02   5.08000e+01   1.57593e-08
   4  Sigma        6.77359e+00   2.98837e-02   2.12001e-03   2.98244e-03


Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc6
